In [ ]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
import numpy as np
 
#Load Data with pandas, and parse the first column into datetime
train=pd.read_csv('C:/Dev/Datos/TP/Data/train.csv', parse_dates = ['Dates'])
test=pd.read_csv('C:/Dev/Datos/TP/Data/test.csv', parse_dates = ['Dates'])

print

In [7]:

# Acá deberíamos incluir también una columna más como la tupla X,Y, lo cual # se hace así(esto agrega una nueva columna a nuestro set, teóricamente de # forma rápida)

train['X_Y'] = zip((-1000*train.X)-122000,(1000*train.Y)-37000 )
test['X_Y'] = zip((-1000*test.X)-122000 , (1000*test.Y)-37000 )

#Convert crime labels to numbers
le_crime = preprocessing.LabelEncoder()
crime = le_crime.fit_transform(train.Category)
 
#Get binarized weekdays, districts, and hours.
days = pd.get_dummies(train.DayOfWeek)
district = pd.get_dummies(train.PdDistrict)
hour = train.Dates.dt.hour
hour = pd.get_dummies(hour)
#Guarda con esto:
coordenada= pd.get_dummies(train.X_Y)
print(coordenada)

0                                        1
1                                        1
2                                        1
3                                        1
4                                        1
5                                        1
6                                        1
7                                        1
8                                        1
9                                        1
10                                       1
11                                       1
12                                       1
13                                       1
14                                       1
15                                       1
16                                       1
17                                       1
18                                       1
19                                       1
20                                       1
21                                       1
22                                       1
23         

In [2]:


#for index, row in coordenada.iterrows():
   # row['X_Y'] = [round(row["X_Y"][0]), round(row["X_Y"][1])]


#Build new array
train_data = pd.concat([hour, days, district,coordenada], axis=1)
train_data['crime']=crime
 
#Repeat for test data
days = pd.get_dummies(test.DayOfWeek)
district = pd.get_dummies(test.PdDistrict)
coordenada= pd.get_dummies(test.X_Y)
 
hour = test.Dates.dt.hour
hour = pd.get_dummies(hour)
 
test_data = pd.concat([hour, days, district, coordenada], axis=1)
 
training, validation = train_test_split(train_data, train_size=.60)
 
features = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
 'Wednesday', 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION',
 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN']
 


features2 = [x for x in range(0,24)]

#[(x, y) for x in [1,2,3] for y in [3,1,4] if x != y]

features3 = [(x,y) for x in range(0,1000) for y in range(0,1000)]
 
features = features + features2 + features3

training, validation = train_test_split(train_data, train_size=.60)
model = BernoulliNB()
model.fit(training[features], training['crime'])
predicted = np.array(model.predict_proba(validation[features]))
log_loss(validation['crime'], predicted)
 
#Logistic Regression for comparison<	
model = LogisticRegression(C=.01)
model.fit(training[features], training['crime'])
predicted = np.array(model.predict_proba(validation[features]))
log_loss(validation['crime'], predicted)
 
model = BernoulliNB()
model.fit(train_data[features], train_data['crime'])
predicted = model.predict_proba(test_data[features])
 
#Write results
result=pd.DataFrame(predicted, columns=le_crime.classes_)
result.to_csv('C:/Dev/Datos/TP/Data/testResult.csv', index = True, index_label = 'Id' )

KeyError: 'X_Y'